In [1]:
import pandas as pd
import numpy as np
import pymc as pm
import seaborn as sns
from sklearn.model_selection import train_test_split
pd.set_option('display.max_rows', 500)
import arviz as az
import matplotlib.pyplot as plt

WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


## READ IN DATA

In [2]:
# select all the players besides dbs and wrs
players = pd.read_csv("https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/players.csv")

positions = ['DE', 'OLB','DT', 'ILB', 'NT', 'MLB', 'LB', 'RB', 'T', 'TE','G','QB','C','FB']
positions_df = players[players['officialPosition'].isin(positions)]
players_list = positions_df['nflId']

In [3]:
# Read in all the weeks but only for the positons above
locations = pd.DataFrame()
for i in range(1,8):
    url = 'https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/week'+str(i)+'.csv'
    week_data = pd.read_csv(url)
    week_data = week_data[(week_data['nflId'].isin(players_list)) | (week_data['team'] == 'football')]
    locations = pd.concat([locations, week_data])

In [4]:
locations

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event
0,2021090900,97,25511.0,1,2021-09-10 00:26:31,12.0,TB,right,37.77,24.22,0.29,0.30,0.03,165.16,84.99,None
1,2021090900,97,25511.0,2,2021-09-10 00:26:31,12.0,TB,right,37.78,24.22,0.23,0.11,0.02,164.33,92.87,None
2,2021090900,97,25511.0,3,2021-09-10 00:26:31,12.0,TB,right,37.78,24.24,0.16,0.10,0.01,160.24,68.55,None
3,2021090900,97,25511.0,4,2021-09-10 00:26:31,12.0,TB,right,37.73,24.25,0.15,0.24,0.06,152.13,296.85,None
4,2021090900,97,25511.0,5,2021-09-10 00:26:31,12.0,TB,right,37.69,24.26,0.25,0.18,0.04,148.33,287.55,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906287,2021102500,3998,NaN,71,2021-10-26 03:17:05,NaN,football,left,112.93,14.52,22.04,0.44,2.62,NaN,NaN,pass_forward
906288,2021102500,3998,NaN,72,2021-10-26 03:17:06,NaN,football,left,111.34,16.07,21.95,1.40,2.22,NaN,NaN,None
906289,2021102500,3998,NaN,73,2021-10-26 03:17:06,NaN,football,left,109.74,17.60,21.81,2.02,2.21,NaN,NaN,None
906290,2021102500,3998,NaN,74,2021-10-26 03:17:06,NaN,football,left,108.17,19.12,21.66,2.39,2.19,NaN,NaN,None


In [ ]:
pff = pd.read_csv("https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/pffScoutingData.csv")
pff

In [ ]:
pff_filter = pff[['gameId','playId','nflId','pff_role','pff_positionLinedUp','pff_nflIdBlockedPlayer','pff_blockType']]

In [ ]:
locations_ball = locations.merge(pff_filter, on=['gameId','playId','nflId'], how='left')

In [ ]:
plays = pd.read_csv("https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/plays.csv")
plays

In [ ]:
# Also going to be added in to the tracking data
plays_filter = plays[['gameId','playId','dropBackType','pff_playAction']]

In [ ]:
locations_ball = locations_ball.merge(plays_filter, on=['gameId','playId'], how='left')
locations_ball

In [ ]:
# create unique play ID for each play
locations_ball['uniqueplayId'] = locations_ball['gameId'].astype(str) + locations_ball['playId'].astype(str)

In [ ]:
# Add the location of the football on the play to each row then standardize x and y on this location
locations_ball = locations_ball.loc[locations_ball['team'] == 'football',
                               ['uniqueplayId', 'x', 'y']
                               ].rename(columns={'x':'football_x',
                                                 'y':'football_y'}).merge(locations_ball, on='uniqueplayId', how='right')
locations_ball['new_x'] = abs(locations_ball['x']-locations_ball['football_x'])
locations_ball['new_y'] = np.where(locations_ball['playDirection']=='right', 
                                   locations_ball['football_y']-locations_ball['y'],
                                   locations_ball['y']-locations_ball['football_y'])